In [1]:
library(data.table)
library(ggplot2)
library(stringr)
library(dplyr)
library(reshape2)
source('../0_support-files/theme_CRP-MISC.R')
source('../0_support-files/prevail_man_fill.R')

'%ni%' <- Negate('%in%')

expGroupPalette <- c("Control" = "#00FF7F",
                            "COVID-19_Nonsevere" = "#FF00FF",
                            "COVID-19_Severe" = "#FF7F00",
                            "Control_Non-inflammatory" ='#76d75b',
                            "Acute COVID-19 Mild/ASX" = '#d42069',
                            "Acute COVID-19 Moderate/Severe" = '#d42069',
                            "Acute MIS-C Moderate/Severe" = '#21aaf8')


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘reshape2’


The following objects are masked from ‘package:data.table’:

    dcast, melt




In [2]:
ID_VARS = c("PTID","Sample_ID","Diagnosis","timepoint","expGroup_plotting","scaled_cfDNA")

TISS_COLS = c('bcell','colon','dendritic','eosinophil','erythroblast','heart','hsc','kidney','liver','lung','macrophage','megakaryocyte','monocyte','neutrophil','nkcell','pancreas','progenitor','skin','spleen','tcell')

XORDER <- c("Control","COVID-19_Nonsevere","COVID-19_Severe","Control_Non-inflammatory","COVID-19","MIS-C")


## Read in MISC data
master_df = read.csv("../1_sample-data/STable8_cfdna-samples.csv") %>% 
    rename(scaled_cfDNA = cfDNA_concentration) %>%
    mutate(severity = ifelse(severity == 0 | severity == 1,"Mild/ASX",
                             ifelse(severity == 2 | severity == 3,"Moderate/Severe",
                                    ""))) %>%
    mutate(Diagnosis = ifelse(Diagnosis != "Control_Non-inflammatory", 
                              paste0("Acute ",Diagnosis," ",severity),
                                     Diagnosis)) %>%
    rename(Sample_ID = cfdna_sample_id) %>% 
    mutate(expGroup_plotting = Diagnosis) %>%
    select(all_of(c(ID_VARS,TISS_COLS)))
                  
## Read in adult cohort
adult_mdf <- read.delim("../1_sample-data/cfdna_adult_master-df.tsv")%>%
    filter(origin == "MCGILL") %>%
    mutate(Diagnosis = severity) %>%
    mutate(expGroup_plotting = severity) %>%
    mutate(icu = 0) %>% 
    mutate(timepoint = "acute") %>%
    select(all_of(c(ID_VARS,TISS_COLS)))
                  
                  
## Combine
TOO_df <- rbind(master_df,adult_mdf)

## Melt and calculate TOO concentration
TOO_df_melt <- melt(TOO_df,id.vars = ID_VARS)
TOO_df_melt$value_abundance = TOO_df_melt$scaled_cfDNA * TOO_df_melt$value

## filter out only relevant columns
TOO_df_melt <- TOO_df_melt

TOO_df_melt$Diagnosis <- factor(TOO_df_melt$Diagnosis,levels=XORDER)

TOO_df_melt$expGroup_plotting <- factor(TOO_df_melt$expGroup_plotting,levels = names(expGroupPalette))

---
## Test all cell types

In [3]:
output <- list()
for (cell in unique(TOO_df_melt$variable)){

    x1 <- TOO_df_melt %>% filter(variable == cell) %>% filter(expGroup_plotting == "Acute MIS-C Moderate/Severe") %>% pull(value)
    x2 <- TOO_df_melt %>% filter(variable == cell) %>% filter(expGroup_plotting == "Acute COVID-19 Moderate/Severe") %>% pull(value)

    w_out <- suppressWarnings(wilcox.test(x1,x2))

    output[[cell]] <- c(cell,w_out$p.value)
    
}
df <- data.frame(do.call("rbind",output))
colnames(df) <- c("cell","p")

df$p.adj <- p.adjust(df$p, method = "BH")

df %>% arrange(p.adj)

,cell,p,p.adj
,<chr>,<chr>,<dbl>
progenitor,progenitor,0.00242466546435291,0.04849331
dendritic,dendritic,0.0273018368121621,0.27301837
megakaryocyte,megakaryocyte,0.0691262060502952,0.46084137
bcell,bcell,0.715995941150521,0.94640700
colon,colon,0.403992307271098,0.94640700
eosinophil,eosinophil,0.946407002882282,0.94640700
erythroblast,erythroblast,0.229374477496456,0.94640700
heart,heart,0.686594885677674,0.94640700
hsc,hsc,0.505808606866995,0.94640700


In [4]:
pdf(file="plots/SupPanelB_too.pdf",
                 width=6.8,height=7.3, paper="special", bg="white",
                 fonts="Helvetica", colormodel = "cmyk", pointsize=6, useDingbats = FALSE)


TOO_df_melt %>% 
ggplot(aes(x=expGroup_plotting,y=value_abundance))+
geom_boxplot(aes(fill = expGroup_plotting), outlier.size = 0.1, size = 0.5,color="black")+
theme_prevail()+
labs(x="",y="cfDNA Abundance (ng/ul)")+
theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust=1),
     legend.position = "none",
     legend.text = element_text(size=20))+
scale_fill_manual(values = expGroupPalette) +
facet_wrap(~ variable,scales = "free_y", ncol = 4)

dev.off()

png 
  2